In [1]:
from datetime import datetime, timedelta
from quantlib.data.service import DataService

In [2]:
# Create service with Alpha Vantage
service = DataService.create_default_service(alpha_vantage_key='1S246WCWWOKHXO01')

# Get data
end_date = datetime.now()
start_date = end_date - timedelta(days=30)

In [3]:
service.list_sources()

['yahoo_finance', 'alpha_vantage', 'mt5']

In [4]:
intraday_data = service.get_ohlc_prices(
    'PETR4',
    'mt5',
    '2025-01-01',
    '2025-01-10',
    interval='1h',
)
intraday_data.data

,open,high,low,close
time,,,,
2025-01-02 10:00:00,36.42,36.50,36.19,36.29
2025-01-02 11:00:00,36.29,36.64,36.21,36.60
2025-01-02 12:00:00,36.61,37.09,36.60,37.03
2025-01-02 13:00:00,37.03,37.03,36.85,36.94
2025-01-02 14:00:00,36.95,36.97,36.87,36.87
2025-01-02 15:00:00,36.87,36.94,36.78,36.92
2025-01-02 16:00:00,36.91,36.95,36.85,36.86
2025-01-02 17:00:00,36.86,36.95,36.82,36.95
2025-01-02 18:00:00,36.77,36.77,36.77,36.77
